In [2]:
from prettytable import PrettyTable
from utils import *


class Node:
    def __init__(self, name):
        self.parent = None   
        self.name = name
        self.edges = []
        self.value = 0 

class Edge:
    def __init__(self, edge):
        self.start = edge[0]
        self.end = edge[1]
        self.value = edge[2]
def getNode(name, l):
   return next(( i for i in l if i.name == name), -1)

class Graph:
    def __init__(self, node_list, edges):
        self.nodes = []
        for name in node_list:
            self.nodes.append(Node(name))
        for e in edges:
            node1 = getNode(e[0], self.nodes)
            node2 = getNode(e[1], self.nodes)
            e = (node1, node2, e[2])
            idx = next((i for i,v in enumerate(self.nodes) if v.name == node1.name), -1)
            self.nodes[idx].edges.append(Edge(e))
            idx2 = next((i for i,v in enumerate(self.nodes) if v.name == node2.name), -1)
            self.nodes[idx2].edges.append(Edge((node2, node1, e[2])))

    def print(self):
        node_list = self.nodes
        t = PrettyTable(['  '] + [i.name for i in node_list])
        for node in node_list:
            edge_values = ['X'] * len(node_list)
            for edge in node.edges:
                pos = next((i for i,e in enumerate(node_list) if e.name == edge.end.name), -1)
                edge_values[pos] = edge.value           
            t.add_row([node.name] + edge_values)
        print(t)


class Queue:
    def __init__(self, mode='FIFO'):
        self.mode = mode 
        self.list = []

    def push(self, item, priority=0):
        if self.mode == 'PRIO':
            self.list.append((priority, item))
            self.list.sort(key=lambda x: x[0])
        else:
            self.list.append(item)

    def pop(self):
        if self.mode == 'FIFO':
            return self.list.pop(0)
        elif self.mode == 'LIFO':
            return self.list.pop()
        elif self.mode == 'PRIO':
            return self.list.pop(0)[1]

    def isEmpty(self):
        return len(self.list) == 0

def reconstruct_path(node):
    path = []
    while node is not None:
        path.append(node.name)
        node = node.parent
    return path[::-1]

# -------------------------------
# Search algorithms
# -------------------------------

def bfs(graph, start_name, goal_name):
    for node in graph.nodes:
        node.parent = None

    start = getNode(start_name, graph.nodes)
    goal  = getNode(goal_name, graph.nodes)

    q = Queue(mode='FIFO')
    q.push(start)
    visited = {start.name}

    while not q.isEmpty():
        current = q.pop()
        if current.name == goal.name:
            return reconstruct_path(current)
        for edge in current.edges:
            if edge.end.name not in visited:
                visited.add(edge.end.name)
                edge.end.parent = current
                q.push(edge.end)
    return None

def dfs(graph, start_name, goal_name):
    for node in graph.nodes:
        node.parent = None

    start = getNode(start_name, graph.nodes)
    goal  = getNode(goal_name, graph.nodes)

    q = Queue(mode='LIFO')
    q.push(start)
    visited = set([start.name])
    
    while not q.isEmpty():
        current = q.pop()
        if current.name == goal.name:
            return reconstruct_path(current)
        for edge in current.edges:
            if edge.end.name not in visited:
                visited.add(edge.end.name)
                edge.end.parent = current
                q.push(edge.end)
    return None

def ucs(graph, start_name, goal_name):
    for node in graph.nodes:
        node.parent = None
        node.value = float('inf')
    start = getNode(start_name, graph.nodes)
    goal  = getNode(goal_name, graph.nodes)
    start.value = 0

    q = Queue(mode='PRIO')
    q.push(start, priority=0)
    visited = {}

    while not q.isEmpty():
        current = q.pop()
        # If we have reached the goal, we found the optimal path.
        if current.name == goal.name:
            return reconstruct_path(current), current.value
        # To avoid re-expanding a node with a higher cost, check if we have a cheaper route.
        for edge in current.edges:
            new_cost = current.value + edge.value
            # If this path to neighbor is better then use it.
            if new_cost < edge.end.value:
                edge.end.value = new_cost
                edge.end.parent = current
                q.push(edge.end, priority=new_cost)
    return None


romania = Graph(
    ['Or', 'Ne', 'Ze', 'Ia', 'Ar', 'Si', 'Fa', 'Va', 'Ri', 'Ti', 'Lu', 'Pi', 'Ur', 'Hi', 'Me', 'Bu', 'Dr', 'Ef', 'Cr', 'Gi'],
    [
      ('Or', 'Ze', 71), ('Or', 'Si', 151), 
      ('Ne', 'Ia', 87), ('Ze', 'Ar', 75),
      ('Ia', 'Va', 92), ('Ar', 'Si', 140),
      ('Ar', 'Ti', 118), ('Si', 'Fa', 99), 
      ('Si', 'Ri', 80), ('Fa', 'Bu', 211),
      ('Va', 'Ur', 142), ('Ri', 'Pi', 97),
      ('Ri', 'Cr', 146), ('Ti', 'Lu', 111),
      ('Lu', 'Me', 70), ('Me', 'Dr', 75),
      ('Dr', 'Cr', 120), ('Cr', 'Pi', 138),
      ('Pi', 'Bu', 101), ('Bu', 'Gi', 90),
      ('Bu', 'Ur', 85), ('Ur', 'Hi', 98), 
      ('Hi', 'Ef', 86)
    ]
)

# Run the searches from Bucharest ('Bu') to Timisoara ('Ti')

bfs_path = bfs(romania, 'Bu', 'Ti')
# Calculate the cost along the path (simply by summing edge values)
def calculate_cost(graph, path):
    cost = 0
    for i in range(len(path)-1):
        node = getNode(path[i], graph.nodes)
        for edge in node.edges:
            if edge.end.name == path[i+1]:
                cost += edge.value
                break
    return cost

bfs_cost = calculate_cost(romania, bfs_path)

# DFS: 
dfs_path = dfs(romania, 'Bu', 'Ti')
dfs_cost = calculate_cost(romania, dfs_path)

# UCS: (Should yield the lowest total cost path)
ucs_result = ucs(romania, 'Bu', 'Ti')
ucs_path, ucs_cost = ucs_result if ucs_result is not None else (None, None)

# -------------------------------
# Print the results
# -------------------------------
print("BFS:")
print("  Path:", bfs_path)
print("  Total cost: {}".format(bfs_cost))
print("\nDFS:")
print("  Path:", dfs_path)
print("  Total cost: {}".format(dfs_cost))
print("\nUCS:")
print("  Path:", ucs_path)
print("  Total cost: {}".format(ucs_cost))


BFS:
  Path: ['Bu', 'Fa', 'Si', 'Ar', 'Ti']
  Total cost: 568

DFS:
  Path: ['Bu', 'Pi', 'Cr', 'Dr', 'Me', 'Lu', 'Ti']
  Total cost: 615

UCS:
  Path: ['Bu', 'Pi', 'Ri', 'Si', 'Ar', 'Ti']
  Total cost: 536


In [3]:
import pygame
import math

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

WIDTH = 22
HEIGHT = 22
MARGIN = 3

WALL = 0
PATH = 1
START = 9

GRID_SIZE = 20


class Node:
    parent = None 
    def __init__(self,walkable = True,x = 0,y = 0):
        self.walkable = walkable
        self.x = x
        self.y = y
        self.g_cost = 0
        self.h_cost =0

    @property
    def f_cost(self):
        return self.g_cost + self.h_cost

    def get_f_cost(self) :
        return self.g_cost + self.h_cost

        
    
class Grid:
    # GRID PATTERN FROM TASK DESCRIPTION
    A = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,], 
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,]]
    path = [] 
    def __init__(self,obsticleGrid = A):
        self.grid = [[None for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.obsticleGrid = obsticleGrid
        for x in range(0,GRID_SIZE) :
            for y in range(0,GRID_SIZE) :
                if self.obsticleGrid[x][y] == PATH :
                    self.grid[x][y] = Node(True,x,y)
                elif self.obsticleGrid[x][y] == WALL :
                    self.grid[x][y] = Node(False,x,y)
                elif self.obsticleGrid[x][y] == START:
                    self.grid[x][y] = Node(True, x, y) 
        pass 
    ## return 3 x 3 neighbours of grid
    def get_neighbours(self, node):
        neighbours = []
        for dx in (-1, 0, 1):
            for dy in (-1, 0, 1):
                if dx == 0 and dy == 0:
                    continue
                nx, ny = node.x + dx, node.y + dy
                if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                    neighbours.append(self.grid[nx][ny])
        return neighbours
    def get_node(self,x,y) :
        return self.grid[x][y]

# ---
## A star
def pathfind(grid, start, target):
    open_set = [start]
    closed_set = []

    while open_set:
        current = min(open_set, key=lambda n: (n.f_cost, n.h_cost))#O(N) use heap for optimaation
        open_set.remove(current)
        closed_set.append(current)

        if current is target:
            return retrace_path(start, target)

        for neigh in grid.get_neighbours(current):
            if not neigh.walkable or neigh in closed_set:
                continue

            tentative_g = current.g_cost + get_distance(current, neigh)
            if tentative_g < neigh.g_cost or neigh not in open_set:
                neigh.g_cost = tentative_g
                neigh.h_cost = get_distance(neigh, target)
                neigh.parent = current
                if neigh not in open_set:
                    open_set.append(neigh)

    return []  # no path found

def pathfind_step_by_step(grid, start, target):
    open_set = [start]
    closed_set = []
    current = None

    while open_set:
        current = min(open_set, key=lambda n: (n.f_cost, n.h_cost))
        open_set.remove(current)
        closed_set.append(current)

        if current is target:
            yield retrace_path(start, target), current, open_set.copy(), closed_set.copy()
            return

        for neigh in grid.get_neighbours(current):
            if not neigh.walkable or neigh in closed_set:
                continue

            tentative_g = current.g_cost + get_distance(current, neigh)
            if tentative_g < neigh.g_cost or neigh not in open_set:
                neigh.g_cost = tentative_g
                neigh.h_cost = get_distance(neigh, target)
                neigh.parent = current
                if neigh not in open_set:
                    open_set.append(neigh)

        yield [], current, open_set.copy(), closed_set.copy()

def retrace_path(start, end):
    path = []
    cur = end
    while cur is not start:
        path.append(cur)
        cur = cur.parent
    path.reverse()
    return path
## to work with full numbers we multiply the distance time 10
## distence between two cells is 1 we set the weight times 10
## distance on diagonal axis is sqrt(2) * 10 is ~14
def get_distance(a, b):
    dx, dy = abs(a.x - b.x), abs(a.y - b.y)
    if dx > dy:
        return 14 * dy + 10 * (dx - dy)
    return 14 * dx + 10 * (dy - dx)

##


labyrinth = [
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    [1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
    [1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
]

pygame.init()

size = (500, 500)
screen = pygame.display.set_mode(size)

pygame.display.set_caption("My Game")

done = False

clock = pygame.time.Clock()

grid = Grid()
#uncomment for normal A star
nodeA = grid.get_node(19,0)
nodeB = grid.get_node(0,19)

#uncomment for step by step a star
#nodeA = grid.get_node(19, 0)
#nodeB = grid.get_node(0, 19)
#step_generator = pathfind_step_by_step(grid, nodeA, nodeB)
#path = []
#current_node = None
#open_set = []
#closed_set = []

pygame.time.set_timer(pygame.USEREVENT, 200)  # every 200ms, advance a step
##
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        ## comment for normal
        #elif event.type == pygame.USEREVENT:
        #    try:
        #        result, current_node, open_set, closed_set = next(step_generator)
        #        if result:
        #            path = result
        #    except StopIteration:
        #        pass
        ##
    ##uncomment
    path = pathfind(grid,nodeA,nodeB)

    screen.fill(BLACK)

    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            node = grid.grid[x][y]
            color = WHITE if node.walkable else BLACK

            if node is nodeA:
                color = GREEN
            elif node is nodeB:
                color = RED
            #elif node == current_node:
            #    color = (255, 165, 0)  # Orange for current node
            elif node in path:
                color = BLUE
            #elif node in open_set:
            #    color = (255, 255, 0)  # Yellow for open set
            #elif node in closed_set:
            #    color = (160, 160, 160)  # Gray for closed set

            pygame.draw.rect(
                screen,
                color,
                [(MARGIN + WIDTH) * y + MARGIN, (MARGIN + HEIGHT) * x + MARGIN, WIDTH, HEIGHT]
            )
            if node.walkable and node.g_cost > 0:
                font = pygame.font.SysFont('Arial', 10)
                text = font.render(str(node.g_cost), True, BLACK)
                screen.blit(text, ((MARGIN + WIDTH) * y + MARGIN + 3,
                                   (MARGIN + HEIGHT) * x + MARGIN + 3))
    

    pygame.display.flip()

    clock.tick(60)

pygame.quit()



pygame 2.6.1 (SDL 2.28.4, Python 3.13.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
